In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV, StratifiedKFold,TimeSeriesSplit,KFold,GroupKFold
from datetime import datetime
from kaggle.competitions import nflrush
import math
import tqdm
from scipy.special import expit
from scipy.spatial import Delaunay, delaunay_plot_2d, Voronoi, voronoi_plot_2d, ConvexHull
pd.set_option("display.max_rows",1000)
env = nflrush.make_env()

In [2]:
def crps_score(y_pred, data):
    y_pred = np.transpose(y_pred.reshape(199,-1))
    y_true = np.array(data.get_label()).astype("int")
    y_true_cdf = np.zeros([y_true.shape[0], 199])
    for i, y in enumerate(y_true):
        y_true_cdf[i, y] = 1
    y_true_cdf = np.clip(np.cumsum(y_true_cdf, axis=1), 0, 1)
    y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
    return "CRPS", ((y_true_cdf - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * y_true.shape[0]), False

def simple_crps(y_true, y_pred):
    y_true_cdf = np.zeros([y_true.shape[0], 199])
    for i, y in enumerate(y_true):
        y_true_cdf[i, y] = 1
    y_true_cdf = np.clip(np.cumsum(y_true_cdf, axis=1), 0, 1)
    y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
    return ((y_true_cdf - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * y_true.shape[0])

In [3]:
train = pd.read_csv('../input/nfl-big-data-bowl-2020/train.csv',low_memory=False)

# preprocess and feature engineering 

In [4]:
def transform_time_all(str1,quarter):
    if quarter<=4:
        return 15*60 - (int(str1[:2])*60 + int(str1[3:5])) + (quarter-1)*15*60
    if quarter ==5:
        return 10*60 - (int(str1[:2])*60 + int(str1[3:5])) + (quarter-1)*15*60
    
def transform_height(te):
    return (int(te.split('-')[0])*12 + int(te.split('-')[1]))*2.54/100

def voronoi_volumes(points, selected_index):
    if np.isnan(points).any(): # if missing value is included
        return -999
    
    else:
        v = Voronoi(points)
        vol = np.zeros(v.npoints)
      
        for i, reg_num in enumerate(v.point_region):
            if reg_num == v.point_region[selected_index]:
                indices = v.regions[reg_num]
                if -1 in indices: # some regions can be opened
                    vol = -999 ## insert missing value when the area is open
                else:
                    vol = ConvexHull(v.vertices[indices]).volume      
                break
        return vol

def radius_calc(dist_to_ball):
    return 4 + 6 * (dist_to_ball >= 15) + (dist_to_ball ** 3) / 560 * (dist_to_ball < 15)

def compute_influence_to_ball(row):
    point = np.array([row["RushX"], row["RushY"]])
    theta = math.radians(row['Orientation'])
    speed = row['S']
    player_coords = row[['X', 'Y']].values
    dist_to_ball = row["DisToRusher"]    

    S_ratio = (speed / 13) ** 2    # we set max_speed to 13 m/s ((dominance_df["S"]*0.9144).max() #8.64m/s)
    RADIUS = radius_calc(dist_to_ball)  # updated

    S_matrix = np.matrix([[RADIUS * (1 + S_ratio), 0], [0, RADIUS * (1 - S_ratio)]])
    R_matrix = np.matrix([[np.cos(theta), - np.sin(theta)], [np.sin(theta), np.cos(theta)]])
    COV_matrix = np.dot(np.dot(np.dot(R_matrix, S_matrix), S_matrix), np.linalg.inv(R_matrix))
    
    norm_fact = (1 / 2 * np.pi) * (1 / np.sqrt(np.linalg.det(COV_matrix)))    
    mu_play = player_coords + speed * np.array([np.cos(theta), np.sin(theta)]) / 2
    
    intermed_scalar_player = np.dot(np.dot((player_coords - mu_play),
                                    np.linalg.inv(COV_matrix)),
                             np.transpose((player_coords - mu_play)))
    player_influence = norm_fact * np.exp(- 0.5 * intermed_scalar_player[0, 0])
    
    intermed_scalar_point = np.dot(np.dot((point - mu_play), 
                                    np.linalg.inv(COV_matrix)), 
                             np.transpose((point - mu_play)))
    point_influence = norm_fact * np.exp(- 0.5 * intermed_scalar_point[0, 0])
    
    return point_influence / player_influence

In [5]:
remove_features = ['GameId','PlayId','DisplayName','GameClock','TimeHandoff','TimeSnap', 'PlayDirection', 'TeamOnOffense', 
                    'PlayerBirthDate', 'is_run', 'NflIdRusher', 'date_game', 'RushX', 'RushY', 'PossessionTeam', 
                   'FieldPosition', 'Position', 'PlayerHeight', 'HomeTeamAbbr', 'VisitorTeamAbbr', 'Turf', 'Quarter'
                   , 'RushX_0.5s', 'RushY_0.5s', 'RushX_1s', 'RushY_1s', 'OffenseFormation', 'Team', 'WindSpeed',
                   'GameWeather', 'WindDirection', 'Location', 'Stadium', 'StadiumType', 'Temperature', 'DefensePersonnel'
                  , 'PlayerCollegeName', 'OffensePersonnel', 'Humidity']

In [6]:
def transform_data(df):
    df.loc[df.VisitorTeamAbbr == "ARI",'VisitorTeamAbbr'] = "ARZ"
    df.loc[df.HomeTeamAbbr == "ARI",'HomeTeamAbbr'] = "ARZ"

    df.loc[df.VisitorTeamAbbr == "BAL",'VisitorTeamAbbr'] = "BLT"
    df.loc[df.HomeTeamAbbr == "BAL",'HomeTeamAbbr'] = "BLT"

    df.loc[df.VisitorTeamAbbr == "CLE",'VisitorTeamAbbr'] = "CLV"
    df.loc[df.HomeTeamAbbr == "CLE",'HomeTeamAbbr'] = "CLV"

    df.loc[df.VisitorTeamAbbr == "HOU",'VisitorTeamAbbr'] = "HST"
    df.loc[df.HomeTeamAbbr == "HOU",'HomeTeamAbbr'] = "HST"

    df['is_run'] = df.NflId == df.NflIdRusher

    df['S'] = 10 * df['Dis']
    df.loc[df.Season == 2017, 'A'] = df[df.Season == 2018]['A'].mean()
    
    df['ToLeft'] = df.PlayDirection == "left"
    df['TeamOnOffense'] = "home"
    df.loc[df.PossessionTeam != df.HomeTeamAbbr, 'TeamOnOffense'] = "away"
    df['OnOffense'] = df.Team == df.TeamOnOffense 
    df['YardLine_std'] = 100 - df.YardLine.copy()
    df.loc[df.FieldPosition.fillna('') == df.PossessionTeam,  
            'YardLine_std'
             ] = df.loc[df.FieldPosition.fillna('') == df.PossessionTeam,  
              'YardLine']
    df['X_std'] = df.X.copy()
    df.loc[df.ToLeft, 'X_std'] = 120 - df.loc[df.ToLeft, 'X'] 
    df['Y_std'] = df.Y.copy()
    df.loc[df.ToLeft, 'Y_std'] = 53.3 - df.loc[df.ToLeft, 'Y'] 
    df['Orientation_std'] = df.Orientation.copy()
    df.loc[df.ToLeft, 'Orientation_std'] = np.mod(180 + df.loc[df.ToLeft, 'Orientation_std'], 360)
    df['Dir_std'] = df.Dir.copy()
    df.loc[df.ToLeft, 'Dir_std'] = np.mod(180 + df.loc[df.ToLeft, 'Dir_std'], 360)
    df.loc[df['Season'] == 2017, 'Orientation_std'] = np.mod(90 + df.loc[df['Season'] == 2017, 'Orientation_std'], 360) 
    df.drop(["X", "Y", "Orientation", "YardLine", "Dir", "ToLeft"], axis=1, inplace=True)
    df.rename(columns={'X_std': 'X', 'Y_std': 'Y', 'Orientation_std': 'Orientation', 'Dir_std': 'Dir', "YardLine_std": "YardLine"}, inplace=True)
    
    df['date_game'] = df.GameId.map(lambda x:pd.to_datetime(str(x)[:8]))
    df['age'] = (df.date_game.map(pd.to_datetime) - df.PlayerBirthDate.map(pd.to_datetime)).map(lambda x:x.days)/365

    df["Momentum"] = df["S"] * df["PlayerWeight"]
    
    df["Dir_cos"] = df["Dir"].apply(lambda x : np.cos((450-x) * np.pi/ 180))
    
    df["Momentum_cos"] = df["Momentum"] * df["Dir_cos"]
    
    df["Dir_sin"] = df["Dir"].apply(lambda x : np.sin((450-x) * np.pi/ 180))
    
    df["S_cos"] = df["S"] * df["Dir_cos"]
    df["S_sin"] = df["S"] * df["Dir_sin"]
    df["X_0.5s"] = df["X"] + df["S_cos"] * 0.5
    df["Y_0.5s"] = df["Y"] + df["S_sin"] * 0.5
    df["X_1s"] = df["X"] + df["S_cos"] * 1
    df["Y_1s"] = df["Y"] + df["S_sin"] * 1

    rusher_x = np.array(df.groupby(["PlayId", "is_run"])["X"].agg(np.mean)[1::2])
    df["RushX"] = np.repeat(rusher_x, 22) # repeat each elemnt 22 times
    rusher_y = np.array(df.groupby(["PlayId", "is_run"])["Y"].agg(np.mean)[1::2])
    df["RushY"] = np.repeat(rusher_y, 22) 
    df["DisToRusher"] = np.sqrt((df["X"] - df["RushX"]) ** 2 + (df["Y"] - df["RushY"]) ** 2)
    df["TackleTimeToRusher"] = df["DisToRusher"] / df["S"] # includes nan when the speed of rusher is 0
    
    rusher_x = np.array(df.groupby(["PlayId", "is_run"])["X_0.5s"].agg(np.mean)[1::2])
    df["RushX_0.5s"] = np.repeat(rusher_x, 22) 
    rusher_y = np.array(df.groupby(["PlayId", "is_run"])["Y_0.5s"].agg(np.mean)[1::2])
    df["RushY_0.5s"] = np.repeat(rusher_y, 22) 
    df["DisToRusher_0.5s"] = np.sqrt((df["X_0.5s"] - df["RushX_0.5s"]) ** 2 + (df["Y_0.5s"] - df["RushY_0.5s"]) ** 2)
    df["TackleTimeToRusher_0.5s"] = df["DisToRusher_0.5s"] / df["S"]

    rusher_x = np.array(df.groupby(["PlayId", "is_run"])["X_1s"].agg(np.mean)[1::2])
    df["RushX_1s"] = np.repeat(rusher_x, 22) 
    rusher_y = np.array(df.groupby(["PlayId", "is_run"])["Y_1s"].agg(np.mean)[1::2])
    df["RushY_1s"] = np.repeat(rusher_y, 22) 
    df["DisToRusher_1s"] = np.sqrt((df["X_1s"] - df["RushX_1s"]) ** 2 + (df["Y_1s"] - df["RushY_1s"]) ** 2)
    df["TackleTimeToRusher_1s"] = df["DisToRusher_1s"] / df["S"]
    
    #df["InfluToBall"] = df.apply(compute_influence_to_ball, axis=1)
    
    df["AttackAngle"] =  np.arctan((df["Y"] - df["RushY"]) / (df["X"] - df["RushX"])) * 180 / np.pi
    
    df_single = df[df.is_run==True].copy()
        
    df_single['time_end'] = df_single.apply(lambda x:transform_time_all(x.loc['GameClock'],x.loc['Quarter']),axis=1)
                                                                                                                                  
    df_single['DefendersInTheBox_vs_Distance'] = df_single['DefendersInTheBox'] / df_single['Distance']                                                           
  
    df_single["DisToQB"] = np.array(df[(df.Position=="QB") | (df.Position=="C")].groupby(["PlayId"]).agg(np.mean)["DisToRusher"])

    df_single['runner_height'] = df_single.PlayerHeight.map(transform_height)
    df_single.drop(remove_features,axis=1,inplace=True) 

    tmp = df.groupby(["PlayId", "OnOffense"]).agg(np.mean)[["X", "Y", "Momentum", "Momentum_cos"]]
    df_single["DefenseAveX"] = np.array(tmp[0::2]["X"])
    df_single["OffenseAveX"] = np.array(tmp[1::2]["X"])
    df_single["DefenseAveY"] = np.array(tmp[0::2]["Y"]) 
    df_single["OffenseAveY"] = np.array(tmp[1::2]["Y"]) 
    df_single["DefenseAveMomentum"] = np.array(tmp[0::2]["Momentum"])
    df_single["OffenseAveMomentum"] = np.array(tmp[1::2]["Momentum"])
    df_single["DefenseAveMomentum_cos"] = np.array(tmp[0::2]["Momentum_cos"])
    df_single["OffenseAveMomentum_cos"] = np.array(tmp[1::2]["Momentum_cos"])

    tmp = df.groupby(["PlayId", "OnOffense"]).agg(["std"])[["X", "Y", "Momentum", "Momentum_cos"]]
    df_single["DefenseStdX"] = np.array(tmp[0::2]["X"])
    df_single["OffenseStdX"] = np.array(tmp[1::2]["X"])
    df_single["DefenseStdY"] = np.array(tmp[0::2]["Y"])
    df_single["OffenseStdY"] = np.array(tmp[1::2]["Y"])
    df_single["DefenseStdMomentum"] = np.array(tmp[0::2]["Momentum"])
    df_single["OffenseStdMomentum"] = np.array(tmp[1::2]["Momentum"])
    df_single["DefenseStdMomentum_cos"] = np.array(tmp[0::2]["Momentum_cos"])
    df_single["OffenseStdMomentum_cos"] = np.array(tmp[1::2]["Momentum_cos"])
    
    tmp = df.groupby(["PlayId", "OnOffense"]).agg(["std"])[["X_0.5s", "Y_0.5s", "X_1s", "Y_1s"]]
    df_single["DefenseStdX_0.5s"] = np.array(tmp[0::2]["X_0.5s"])
    df_single["OffenseStdX_0.5s"] = np.array(tmp[1::2]["X_0.5s"])
    df_single["DefenseStdY_0.5s"] = np.array(tmp[0::2]["Y_0.5s"]) 
    df_single["OffenseStdY_0.5s"] = np.array(tmp[1::2]["Y_0.5s"]) 
    df_single["DefenseStdX_1s"] = np.array(tmp[0::2]["X_1s"])
    df_single["OffenseStdX_1s"] = np.array(tmp[1::2]["X_1s"])
    df_single["DefenseStdY_1s"] = np.array(tmp[0::2]["Y_1s"]) 
    df_single["OffenseStdY_1s"] = np.array(tmp[1::2]["Y_1s"])
    
    df_single["RunnerToDefenseCentoid"] = np.sqrt((df_single["X"] - df_single["DefenseAveX"]) ** 2 + (df_single["Y"] - df_single["DefenseAveY"]) ** 2)
    df_single["RunnerToOffenseCentoid"] = np.sqrt((df_single["X"] - df_single["OffenseAveX"]) ** 2 + (df_single["Y"] - df_single["OffenseAveY"]) ** 2)
    
    df_single["RunnerToScrimmage"] = df_single["X"] - (df_single["YardLine"] + 10)

    df_single["MinTackleTime"] = np.array(df.groupby(["PlayId", "OnOffense"])["TackleTimeToRusher"].min()[0::2])
    df_single["MinTackleTime_0.5s"] = np.array(df.groupby(["PlayId", "OnOffense"])["TackleTimeToRusher_0.5s"].min()[0::2])
    df_single["MinTackleTime_1s"] = np.array(df.groupby(["PlayId", "OnOffense"])["TackleTimeToRusher_1s"].min()[0::2])
    df_single["1stDefender_Momentum_cos"] = np.array(df.loc[df.groupby(["PlayId", "OnOffense"])["DisToRusher"].idxmin()[0::2]]["Momentum_cos"])
    first_offenser_index =df.groupby(["PlayId", "OnOffense"])["DisToRusher"].nsmallest(2)[3::4].reset_index()["level_2"]
    df_single["1stOffenser_Momentum_cos"] = np.array(df.loc[first_offenser_index]["Momentum_cos"])
    df_single["1stDefender_AttackAngle"] = np.array(df.loc[df.groupby(["PlayId", "OnOffense"])["DisToRusher"].idxmin()[0::2]]["AttackAngle"])    
    
    #def_influ = np.array(df.groupby(["PlayId", "OnOffense"]).agg("sum")["InfluToBall"][0::2])
    #off_influ = np.array(df.groupby(["PlayId", "OnOffense"]).agg("sum")["InfluToBall"][1::2])
    #df_single["PitchControl"] = expit(off_influ - def_influ)
    
    df_single.fillna(-999,inplace=True) 
    remove_features2 = ["OnOffense", "DisToRusher", "TackleTimeToRusher", "OffenseAveX", "OffenseAveY", "AttackAngle", "Dis", 
                    "DisToRusher_0.5s", "TackleTimeToRusher_0.5s", "DisToRusher_1s", "TackleTimeToRusher_1s"]
    df_single.drop(remove_features2, axis=1, inplace=True)

    return df_single

In [7]:
%%time
train_single = transform_data(train)
y_train = train_single.Yards + 99 # to categorize
X_train = train_single.drop(['Yards'],axis=1)
for f in X_train.columns:
    if X_train[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f])+[-999])
        X_train[f] = lbl.transform(list(X_train[f]))

/opt/conda/lib/python3.6/site-packages/pandas/core/series.py:856: RuntimeWarning: invalid value encountered in arctan
  result = getattr(ufunc, method)(*inputs, **kwargs)


CPU times: user 5min 16s, sys: 2.62 s, total: 5min 18s
Wall time: 5min 15s


# modelling

In [8]:
# modified CV
df_20172018 = pd.concat([X_train, y_train], axis=1)
df_2017 = df_20172018[df_20172018.Season==2017].copy()
df_2018 = df_20172018[df_20172018.Season==2018].copy()
X_2017 = df_2017.drop(['Yards'],axis=1)
y_2017 = df_2017.Yards
X_2018 = df_2018.drop(['Yards'],axis=1)
y_2018 = df_2018.Yards
groups = np.array(X_2018.Week)
n_folds=3
gkf=GroupKFold(n_splits = n_folds)

average_crps = 0
models = []
crps_scores = []
lgbm_params = {
    "objective" : "multiclass",
    "metric" : "None", # if set None, only custom metric is considered.
    "tree_learner": "serial",
    "max_depth" : 5,
    "boosting": 'gbdt',
    "num_class": 199,
    "num_leaves" : 13,
    "learning_rate" : 0.01,
    #"bagging_freq": 5,
    #"bagging_fraction" : 0.4,
    #"feature_fraction" : 0.05,
    #"min_data_in_leaf": 80,
}
evals_result = {}
feature_importance_df = pd.DataFrame(list(X_train.columns), columns=["Feature"])
for i , (train_index, test_index) in enumerate(gkf.split(X_2018, y_2018, groups)):
    X_train_2018 = X_2018.iloc[train_index,:]
    y_train_2018 = y_2018.iloc[train_index]
    X_test2 = X_2018.iloc[test_index,:]
    y_test2 = y_2018.iloc[test_index]
    X_train2 = pd.concat([X_2017, X_train_2018])
    y_train2 = pd.concat([y_2017, y_train_2018])
    lgb_train = lgb.Dataset(X_train2, y_train2)
    lgb_eval = lgb.Dataset(X_test2, y_test2, reference=lgb_train)
    
    clf = lgb.train(
        lgbm_params, lgb_train,
        valid_sets=lgb_eval,
        num_boost_round=100000,
        early_stopping_rounds=10,
        evals_result=evals_result,
        feval=crps_score,
    )
    
    models.append(clf)
    temp_predict = clf.predict(X_test2, num_iteration = clf.best_iteration)
    score = simple_crps(y_test2, temp_predict)
    print(score)
    crps_scores.append(score)
    average_crps += score / n_folds
    feature_importance_df["Fold_"+str(i+1)] = clf.feature_importance()
    
feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]
print('crps list:', crps_scores)
print('average crps:',average_crps)

[1]	valid_0's CRPS: 0.0146357
Training until validation scores don't improve for 10 rounds
[2]	valid_0's CRPS: 0.0146274
[3]	valid_0's CRPS: 0.0146192
[4]	valid_0's CRPS: 0.0146114
[5]	valid_0's CRPS: 0.0146033
[6]	valid_0's CRPS: 0.0145953
[7]	valid_0's CRPS: 0.0145873
[8]	valid_0's CRPS: 0.0145794
[9]	valid_0's CRPS: 0.0145716
[10]	valid_0's CRPS: 0.0145643
[11]	valid_0's CRPS: 0.0145566
[12]	valid_0's CRPS: 0.0145487
[13]	valid_0's CRPS: 0.0145414
[14]	valid_0's CRPS: 0.0145338
[15]	valid_0's CRPS: 0.0145265
[16]	valid_0's CRPS: 0.0145192
[17]	valid_0's CRPS: 0.0145123
[18]	valid_0's CRPS: 0.014505
[19]	valid_0's CRPS: 0.0144977
[20]	valid_0's CRPS: 0.0144906
[21]	valid_0's CRPS: 0.0144837
[22]	valid_0's CRPS: 0.0144769
[23]	valid_0's CRPS: 0.0144701
[24]	valid_0's CRPS: 0.014463
[25]	valid_0's CRPS: 0.0144562
[26]	valid_0's CRPS: 0.0144495
[27]	valid_0's CRPS: 0.0144428
[28]	valid_0's CRPS: 0.0144364
[29]	valid_0's CRPS: 0.0144298
[30]	valid_0's CRPS: 0.0144234
[31]	valid_0's CRPS:

- adding distoQB05s, distoQB1s: becomes worse
- 
- removing offense formation
- crps list: [0.013632101693606397, 0.01358508403732003, 0.013520692582067362]
- average crps: 0.013579292770997928
- 
- removing team, windspeed, removing gameweather, winddirection, grass, stadium, location, rushervoronoi, stadiumtype
- crps list: [0.013616662760921272, 0.013577871329087611, 0.013507607302042283]
- average crps: 0.013567380464017054
- 
- adding firstdefender voronoi 1s: becomes worse in 2 out of 3
- 
- removing firstdefender voronoi
- crps list: [0.013621518106336437, 0.013568990856361466, 0.01350682869798734]
- average crps: 0.013565779220228415
- 
- removing temparature
- crps list: [0.013614209837507416, 0.013567753501590552, 0.013508431894902197]
- average crps: 0.013563465078000055
- 
- removing defensepersonnel
- crps list: [0.013617778020657154, 0.013564372513792245, 0.013505837796825853]
- average crps: 0.013562662777091752
- 
- removing playercollegename
- crps list: [0.01361703860498166, 0.013562654406126414, 0.013499162767791648]
- average crps: 0.013559618592966574
- 
- removing offensepersonnel
- crps list: [0.013610294527536378, 0.013559974312122601, 0.013502674720453332]
- average crps: 0.013557647853370769
- 
- removing humidity
- crps list: [0.013612910727428224, 0.013553843652813499, 0.013500208226488583]
- average crps: 0.013555654202243434
- 
- jerseynumber, down: not remove

In [9]:
X_train.columns

Index(['S', 'A', 'NflId', 'JerseyNumber', 'Season', 'Down', 'Distance',
       'HomeScoreBeforePlay', 'VisitorScoreBeforePlay', 'DefendersInTheBox',
       'PlayerWeight', 'Week', 'YardLine', 'X', 'Y', 'Orientation', 'Dir',
       'age', 'Momentum', 'Dir_cos', 'Momentum_cos', 'Dir_sin', 'S_cos',
       'S_sin', 'X_0.5s', 'Y_0.5s', 'X_1s', 'Y_1s', 'time_end',
       'DefendersInTheBox_vs_Distance', 'DisToQB', 'runner_height',
       'DefenseAveX', 'DefenseAveY', 'DefenseAveMomentum',
       'OffenseAveMomentum', 'DefenseAveMomentum_cos',
       'OffenseAveMomentum_cos', 'DefenseStdX', 'OffenseStdX', 'DefenseStdY',
       'OffenseStdY', 'DefenseStdMomentum', 'OffenseStdMomentum',
       'DefenseStdMomentum_cos', 'OffenseStdMomentum_cos', 'DefenseStdX_0.5s',
       'OffenseStdX_0.5s', 'DefenseStdY_0.5s', 'OffenseStdY_0.5s',
       'DefenseStdX_1s', 'OffenseStdX_1s', 'DefenseStdY_1s', 'OffenseStdY_1s',
       'RunnerToDefenseCentoid', 'RunnerToOffenseCentoid', 'RunnerToScrimmage',
       '

In [10]:
feature_importance_df.sort_values("Average").head(100).reset_index(drop=True)

,Feature,Fold_1,Fold_2,Fold_3,Average,Std,Cv
0,Season,865,1038,557,820.000000,198.928798,0.242596
1,Down,1634,1926,2465,2008.333333,344.213435,0.171393
2,DefendersInTheBox,3846,4472,4665,4327.666667,349.584833,0.080779
3,X_0.5s,4798,5999,4519,5105.333333,642.100892,0.125771
4,DefenseStdY_0.5s,4428,5166,5829,5141.000000,572.228975,0.111307
5,runner_height,5177,6778,6967,6307.333333,802.982081,0.127309
6,S_sin,6372,6826,7524,6907.333333,473.805398,0.068595
7,Week,5651,8215,7120,6995.333333,1050.453976,0.150165
8,Distance,7636,5075,8430,7047.000000,1431.595148,0.203150
9,OffenseStdY,4234,7946,9534,7238.000000,2220.877904,0.306836


# prediction

In [11]:
for (test_df, sample_prediction_df) in env.iter_test():
    X_test = transform_data(test_df)
    for f in X_test.columns:
        if X_test[f].dtype=='object':
            X_test[f] = X_test[f].map(lambda x:x if x in set(X_train[f]) else -999)
    for f in X_test.columns:
        if X_test[f].dtype=='object': 
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(X_train[f])+[-999])
            X_test[f] = lbl.transform(list(X_test[f])) 
    pred_value = 0
    for model in models:
        pred_value += model.predict(X_test, num_iteration = model.best_iteration)[0]/5
    pred_data = np.clip(np.cumsum(pred_value), 0, 1)
    pred_data = np.array(pred_data).reshape(1,199)
    pred_target = pd.DataFrame(index = sample_prediction_df.index, \
                               columns = sample_prediction_df.columns, \
                               data = pred_data)
    env.predict(pred_target)
env.write_submission_file()

Your submission file has been saved!  Once you `Commit` your Notebook and it finishes running, you can submit the file to the competition from the Notebook Viewer `Output` tab.
